# GSC Query Analysis - August 2024 

In [ ]:
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from datetime import datetime
from fpdf import FPDF
from PyPDF2 import PdfFileMerger
from collections import defaultdict, Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk







In [ ]:
# Step 1: Get branded terms and extract grams, bigrams, and trigrams
brand_name = input("What is your full brand name? ")
brand_name_lower = brand_name.lower()
brand_words = brand_name_lower.split()
brand_bigrams = [f"{brand_words[i]} {brand_words[i+1]}" for i in range(len(brand_words)-1)]
brand_trigrams = [f"{brand_words[i]} {brand_words[i+1]} {brand_words[i+2]}" for i in range(len(brand_words)-2)]

# Combine all branded terms into a single set
excluded_terms = set(brand_words + brand_bigrams + brand_trigrams)

print(f"Excluded terms list created: {excluded_terms}")


# Step 1: Get brand name and generate grams
#brand_name = input("What is your full brand name? ")
#brand_name_lower = brand_name.lower()
#brand_words = brand_name_lower.split()
#brand_bigrams = [f"{brand_words[i]} {brand_words[i+1]}" for i in range(len(brand_words)-1)]
#brand_trigrams = [f"{brand_words[i]} {brand_words[i+1]} {brand_words[i+2]}" for i in range(len(brand_words)-2)]

#print(f"Grams identified: {', '.join(brand_words)}")
#print(f"Bigrams identified: {', '.join(brand_bigrams)}")
#print(f"Trigrams identified: {', '.join(brand_trigrams)}")

In [ ]:
# Step 2: Accept non-branded terms
#non_branded_terms = set()
#print("Are any of these terms non-branded? Enter STOP to stop.")
#while True:
    #term = input("Enter a term: ").strip().lower()
    #if term == 'stop':
        #break
    #if term in brand_words or any(term in bigram for bigram in brand_bigrams) or any(term in trigram for trigram in brand_trigrams):
        #print("This term is not one of the branded terms you gave before - try again or STOP!")
    #else:
        #non_branded_terms.add(term)


# Step 2: Ask for exceptions to the excluded terms list
exceptions = set()
print("Enter exceptions to the excluded terms list. Enter STOP to stop.")
while True:
    term = input("Enter a term: ").strip().lower()
    if term == 'stop':
        break
    
    # Check if the term is an exact match with any of the terms in the excluded list
    if term in excluded_terms:
        exceptions.add(term)
        print(f"Term '{term}' added as an exception for analysis.")
    else:
        print("This term is not in the excluded terms list. Try again.")

print("Exceptions identified:", exceptions)

# You can now use 'excluded_terms' to filter queries and 'exceptions' to include in the analysis



In [ ]:
# Step 3: Load the Excel file
# make sure you have the queries in column A of the "query" tab i.e. standard GSC export
file_path = 'path-to-your-excel-download-of-gsc-data-here.xlsx'
df = pd.read_excel(file_path, sheet_name='Queries')

# Clean up queries
def clean_query(query):
    query = query.lower()
    for term in brand_words + brand_bigrams + brand_trigrams:
        query = re.sub(r'\b' + re.escape(term) + r'\b', '', query)
    query = re.sub(r'\s+', ' ', query).strip()
    return query

df['Cleaned Queries'] = df['Top queries'].apply(clean_query)

In [ ]:
# Co-occurrence Matrix
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['Cleaned Queries'])
X_array = X.toarray()
co_occurrence_matrix = np.dot(X_array.T, X_array)
co_occurrence_df = pd.DataFrame(co_occurrence_matrix, index=vectorizer.get_feature_names_out(), columns=vectorizer.get_feature_names_out())

In [ ]:
# Plot Co-occurrence Matrix
plt.figure(figsize=(12, 10))
sns.heatmap(co_occurrence_df, cmap='viridis', annot=False, fmt='d')
plt.title('Co-occurrence Matrix')
heatmap_path = f"{brand_name} GSC research {datetime.now().strftime('%Y-%m-%d')}.pdf"
plt.savefig(heatmap_path, format='pdf')
plt.close()

In [ ]:
# Word Cloud
text = ' '.join(df['Cleaned Queries'])
wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=set(vectorizer.get_stop_words())).generate(text)

In [ ]:
# Plot Word Cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
wordcloud_path = f"{brand_name} GSC research {datetime.now().strftime('%Y-%m-%d')}.pdf"
plt.savefig(wordcloud_path, format='pdf')
plt.close()

In [ ]:
#define paths for output PNG files
heatmap_path = f"{brand_name} GSC research {datetime.now().strftime('%Y-%m-%d')}_heatmap.png"
wordcloud_path = f"{brand_name} GSC research {datetime.now().strftime('%Y-%m-%d')}_wordcloud.png"

# Ensure necessary downloads
nltk.download('punkt')
nltk.download('stopwords')

# Step 1: Read the Excel file
file_path = 'heroeslawncare.com-queries-meeting-followup-aug-5-2024.xlsx'
df = pd.read_excel(file_path, sheet_name='Queries')

# Step 2: Combine the relevant text columns (assuming 'Top queries' contains the text data)
combined_text = ' '.join(df['Top queries'].astype(str))

# Step 3: Define additional stopwords for this analysis
brand_name = "Heroes Lawn Care Lawncare"  # Replace with your actual brand name
brand_name_lower = brand_name.lower()
brand_words = brand_name_lower.split()
additional_excluded_keywords = brand_words + [
    'one', 'would', 'like', 'is', 'i', 'hello', 
    'heroes', 'lawn', 'care', 'lawncare', 'heroeslawncare.com'
]

# Step 4: Tokenize and filter the text data
stop_words = set(stopwords.words('english'))
tokens = word_tokenize(combined_text)
filtered_tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stop_words and word.lower() not in additional_excluded_keywords]

# Step 5: Get the top 20 keywords
top_20_keywords = [item[0] for item in Counter(filtered_tokens).most_common(20)]

# Step 6: Calculate the co-occurrence matrix for the top 20 keywords
co_occurrence_matrix = defaultdict(lambda: defaultdict(int))
window_size = 5

for i in range(len(filtered_tokens) - window_size + 1):
    window = filtered_tokens[i:i+window_size]
    for j, word1 in enumerate(window):
        if word1 in top_20_keywords:
            for k in range(j+1, len(window)):
                word2 = window[k]
                if word2 in top_20_keywords:
                    co_occurrence_matrix[word1][word2] += 1
                    co_occurrence_matrix[word2][word1] += 1

# Step 7: Convert the co-occurrence matrix to a pandas DataFrame
co_occurrence_df = pd.DataFrame(co_occurrence_matrix).fillna(0)

# Step 8: Visualize the co-occurrence matrix using a heatmap with annotations
heatmap_path = f"{brand_name} GSC research {datetime.now().strftime('%Y-%m-%d')}_heatmap.png"
plt.figure(figsize=(12, 10))
sns.heatmap(co_occurrence_df, annot=True, cmap='YlGnBu', fmt='d', annot_kws={"size": 8}, cbar_kws={'label': 'Frequency'})
plt.title('Keyword Co-occurrence Matrix')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()  # Ensure everything fits without overlap
plt.savefig(heatmap_path, format='png')
plt.close()

# Save word cloud as PNG
wordcloud_path = f"{brand_name} GSC research {datetime.now().strftime('%Y-%m-%d')}_wordcloud.png"
wordcloud_text = ' '.join(filtered_tokens)
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(wordcloud_text)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.savefig(wordcloud_path, format='png')
plt.close()

# Create PDF with FPDF
pdf = FPDF()

# Add Co-occurrence Matrix page
pdf.add_page()
pdf.set_font("Arial", size = 12)
pdf.cell(200, 10, txt = "Co-occurrence Matrix", ln = True, align = 'C')

# Add refined description for Co-occurrence Matrix
pdf.set_font("Arial", size = 10)
pdf.ln(10)  # Add a line break for spacing
pdf.multi_cell(0, 10, txt="A co-occurrence matrix displays the frequency with which pairs of top keywords appear together within a specified window of text. Each cell in the matrix represents the number of times a specific pair of keywords co-occurs in the data, helping to identify common word associations.")

# Add the co-occurrence matrix image
pdf.image(heatmap_path, x = 10, y = 60, w = 180)  # Adjust y and w as needed

# Add Word Cloud page
pdf.add_page()
pdf.set_font("Arial", size = 12)
pdf.cell(200, 10, txt = "Word Cloud", ln = True, align = 'C')

# Add description for Word Cloud
pdf.set_font("Arial", size = 10)
pdf.ln(10)  # Add a line break for spacing
pdf.multi_cell(0, 10, txt="A word cloud is a visual representation of words where the size of each word indicates its frequency or importance. In this word cloud, larger words appear more frequently in the data, highlighting the most common terms.")

# Add the word cloud image
pdf.image(wordcloud_path, x = 10, y = 60, w = 180)  # Adjust y and w as needed

# Save the combined PDF
combined_pdf_path = f"{brand_name} GSC research {datetime.now().strftime('%Y-%m-%d')}.pdf"
pdf.output(combined_pdf_path)

print(f"PDF saved as {combined_pdf_path}")

In [ ]:
# Clean up intermediate files
import os
os.remove(heatmap_path)
os.remove(wordcloud_path)